In [1]:
import pandas as pd
import numpy as np
class Nueral_Network:
    def __init__(self,dataset,y_test,nueron=[3,1],opti='momentum',weight_init='He',type='regression',learning_rate=0.01,epoch=1000):
        self.rate = learning_rate
        self.count=0
        self.nueron=nueron
        self.class_index = {}
        self.der_z={}
        self.der_b={}
        self.der_w={}
        self.velocity={}
        self.m_velocity={}
        self.bais_velocity={}
        self.m_bais = {}
        self.v_bais={}
        
        if(opti=='momentum'):
            self.update_parameter = self.momentum_update
        elif(opti=='Adam'):
            self.update_parameter=self.adam_update
        self.dataset = dataset
        
        if(nueron[-1]>1):
            
            self.target=self.one_hot_coding(y_test.values)
            
        else:
            self.target = y_test.values.reshape(-1,1)
            
        self.layer=len(nueron)
        self.node = nueron
        
        self.bais_velocity['V1']=np.zeros((1,self.node[0]))
        self.velocity['V1']=np.zeros((self.node[0],self.dataset.shape[1]))
        self.m_velocity['m1']=np.zeros((self.node[0],self.dataset.shape[1]))

        self.m_bais['mb1']=np.zeros((1,self.node[0]))
        self.v_bais['vb1']=np.zeros((1,self.node[0]))


        
        for i in range(2,self.layer+1):
            self.velocity['V'+str(i)]=np.zeros((self.node[i-1],self.node[i-2]))
            self.m_velocity['m'+str(i)]=np.zeros((self.node[i-1],self.node[i-2]))
            
            self.bais_velocity['V'+str(i)]=np.zeros((1,self.node[i-1]))
            self.m_bais['mb'+str(i)]=np.zeros((1,self.node[i-1]))
            self.v_bais['vb'+str(i)]=np.zeros((1,self.node[i-1]))
            
            

        
        if(nueron[-1]>1):
            self.activation=self.softmax
            self.loss=self.softmax_loss
            self.last_der = self.softmax_last_der
            
        elif(nueron[-1]==1 and type=='regression'):
            self.activation=self.regression
            self.loss=self.regression_loss
            self.last_der = self.regression_last_der

        elif(nueron[-1]==1 and type=='binary'):
            self.activation=self.sigmoid
            self.loss=self.sigmoid_loss
            self.last_der = self.sigmoid_last_der
            
        if(weight_init=='Xavier'):
            self.weights,self.bais = self.Xavier()
            
        elif(weight_init=='He'):
            self.weights,self.bais = self.He()
            
        for i in range(epoch):
            self.z,self.a = self.forward()
            y_hat = self.a['A'+str(self.layer)]
            loss_cal = self.loss(y_hat)
            #===========================backpropagation==============
            self.der_z['dz'+str(self.layer)]=self.last_der(y_hat)
            self.der_w['dw'+str(self.layer)]=(1/self.dataset.shape[0])*(np.dot((self.der_z['dz'+str(self.layer)]).T,(self.a['A'+str(self.layer-1)])))
            self.der_b['db'+str(self.layer)] = (1/self.dataset.shape[0])*np.sum(self.der_z['dz'+str(self.layer)],axis=0,keepdims=True)
           #======================last_layer_ko_chhod_kar================
            for k in reversed(range(1,self.layer)):
                self.der_z['dz'+str(k)]=(np.dot((self.der_z['dz'+str(k+1)]),(self.weights['W'+str(k+1)])))*self.der_relu(self.z['Z'+str(k)])
                self.der_w['dw'+str(k)]=(1/self.dataset.shape[0])*np.dot(self.der_z['dz'+str(k)].T,self.a['A'+str(k-1)])
                self.der_b['db'+str(k)] = (1/self.dataset.shape[0])*np.sum(self.der_z['dz'+str(k)],axis=0,keepdims=True)
            #======================Updating_parameter==================
            self.update_parameter()
            if(i%100==0):
                print(f'Loss after {i} iteration is {loss_cal}')

        print(f"Final loss is {loss_cal}")
        print('''
        (\___/)
        (^ _ ^)
        (>Training complete💕
        ''')
        
        

    def one_hot_coding(self,y_test):
        y_test = np.array(y_test)
        
        pos = 0
        for i in y_test:
            if i not in self.class_index:
                self.class_index[i]=pos
                pos = pos+1
        target = []
        for i in y_test:
            new = []
            for j in range(self.nueron[-1]):
                if(j==self.class_index[i]):
                    new.append(1)
                else:
                    new.append(0)
            target.append(new)
        return np.array(target)
        
    def adam_update(self):
        beta1=0.9
        beta2=0.99
        eps=1e-8
        self.count = self.count+1
        
        for i in range(1,self.layer+1):
            self.m_velocity['m'+str(i)]=beta1*self.m_velocity['m'+str(i)]+(1-beta1)*self.der_w['dw'+str(i)]
            self.velocity['V'+str(i)]=beta2*self.velocity['V'+str(i)]+(1-beta2)*((self.der_w['dw'+str(i)])**2)
            self.weights['W'+str(i)]=self.weights['W'+str(i)]-((self.rate)/((self.velocity['V'+str(i)]+eps)**0.5))*self.m_velocity['m'+str(i)]

            self.m_bais['mb'+str(i)]= beta1*self.m_bais['mb'+str(i)]+(1-beta1)*(self.der_b['db'+str(i)])
            self.v_bais['vb'+str(i)]= beta2*self.v_bais['vb'+str(i)]+(1-beta2)*((self.der_b['db'+str(i)])**2)

            mb_hat = self.m_bais['mb'+str(i)] / (1 - beta1 ** self.count)
            vb_hat = self.v_bais['vb'+str(i)] / (1 - beta2 ** self.count)
            self.bais['b'+str(i)] = self.bais['b'+str(i)]-(self.rate * mb_hat / (np.sqrt(vb_hat) + eps))
            
            



            
            
        
        
    def momentum_update(self):
        for i in range(1,self.layer+1):
            
            self.velocity['V'+str(i)]=0.9*self.velocity['V'+str(i)]+(self.rate)*self.der_w['dw'+str(i)]
            self.weights['W'+str(i)] = self.weights['W'+str(i)]-self.velocity['V'+str(i)]

            self.bais_velocity['V'+str(i)]=0.9*self.bais_velocity['V'+str(i)]+(self.rate)*self.der_b['db'+str(i)]
            self.bais['b'+str(i)] = self.bais['b'+str(i)]-self.bais_velocity['V'+str(i)]
            
        
    def predict(self,test_dataset):
        test_dataset = np.array(test_dataset)
        if (test_dataset.ndim==1):
            test_dataset = test_dataset.reshape(1,-1)
        input = test_dataset
        for i in range(1,self.layer):
            z=np.dot(input,self.weights['W'+str(i)].T)+self.bais['b'+str(i)]
            a = self.relu(z)
            input =a
        z = np.dot(input,self.weights['W'+str(self.layer)].T)+self.bais['b'+str(self.layer)]
        output = self.activation(z)
        output = np.array(output)
        
        if(self.activation==self.sigmoid):
            #print("I am inside activation")
            result =[]
            
            for i in range(len(output)):
                if(output[i]>0.5):
                    result.append(1)
                else:
                    result.append(0)
            return result
            
        if(self.activation==self.softmax):
            
            result= np.argmax(output,axis=1)
            final_result=[]
            reverse_dic = {}
            
            for key,value in self.class_index.items():
                reverse_dic[value]=key
            
            for i in result:
                final_result.append(reverse_dic[i])
                
            return final_result
            
        return output
        
            
    def forward(self): 
        z={}
        a={}
        a['A0']=self.dataset
        for i in range(1,self.layer):
            z['Z'+str(i)]=np.dot(a['A'+str(i-1)],self.weights['W'+str(i)].T)+self.bais['b'+str(i)]
            a['A'+str(i)]=self.relu(z['Z'+str(i)])
        z['Z'+str(self.layer)]=np.dot(a['A'+str(self.layer-1)],self.weights['W'+str(self.layer)].T)+self.bais['b'+str(self.layer)]
        a['A'+str(self.layer)]=self.activation(z['Z'+str(self.layer)])
        return z,a

    def sigmoid(self,value):
        value = np.clip(value,-500,500)
        return 1/(1+np.exp(-value))

    def sigmoid_loss(self,y_hat):
        eps = 1e-8
        loss = -(self.target*(np.log(y_hat+eps))+(1-self.target)*(np.log(1-y_hat+eps)))
        return np.mean(loss)
    
    def sigmoid_last_der(self,y_hat):
        return y_hat-self.target
        
    def regression(self,value):
        return value
        
    def regression_loss(self,y_hat):
        #y_hat = np.clip(y_hat, -1e6, 1e6)
        loss = (y_hat-self.target)**2
        return np.mean(loss)
        
    def regression_last_der(self,y_hat):
        
        last=-2*(self.target-y_hat)
        return last
        

        
    def softmax(self,z):
        z = z - np.max(z, axis=1, keepdims=True)
        exp_z = np.exp(z)
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
        
        
    def softmax_loss(self,y_hat):
        eps = 1e-8
        y_hat = np.clip(y_hat, eps, 1 - eps)
        loss = -(np.sum(self.target*(np.log(y_hat)),axis=1))
        return np.mean(loss)
        
    def softmax_last_der(self,y_hat):
        return y_hat-self.target
        
    def relu(self,z):
        return np.maximum(0,z)
        
    def der_relu(self,z):
        return (z > 0).astype(float)
        
    def Xavier(self):
        weights = {}
        bais = {}
        weights['W1']=np.random.randn(self.node[0],self.dataset.shape[1])*((6/(self.node[0]+self.dataset.shape[1]))**0.5)
        bais['b1']=np.zeros((1,self.node[0]))
        for i in range(2,self.layer+1):
            weights['W'+str(i)]=np.random.randn(self.node[i-1],self.node[i-2])*((6/(self.node[i-1]+self.node[i-2]))**0.5)
            bais['b'+str(i)]=np.zeros((1,self.node[i-1]))
        return weights,bais
    def He(self):
        weights = {}
        bais = {}
        weights['W1']=np.random.randn(self.node[0],self.dataset.shape[1])*((2/(self.dataset.shape[1]))**0.5)
        bais['b1']=np.zeros((1,self.node[0]))
        for i in range(2,self.layer+1):
            weights['W'+str(i)]=np.random.randn(self.node[i-1],self.node[i-2])*((2/(self.node[i-2]))**0.5)
            bais['b'+str(i)]=np.zeros((1,self.node[i-1]))
        return weights,bais

        


            
        

<>:95: SyntaxWarning: invalid escape sequence '\_'
<>:95: SyntaxWarning: invalid escape sequence '\_'
/var/folders/qx/m587h9hn4155cbn53_ttrdhc0000gn/T/ipykernel_2629/3735593953.py:95: SyntaxWarning: invalid escape sequence '\_'
  (\___/)


In [2]:
def macro_f1(y_pre,y_test):
    f1_scores =[]
    unique_value=np.unique(y_test)
    for value in unique_value:
        true_pos = np.sum((y_pre==value)&(y_test==value))
        false_pos = np.sum((y_pre==value)&(y_test!=value))
        false_neg = np.sum((y_pre!=value)&(y_test==value))
        true_neg =np.sum((y_pre!=value)&(y_test!=value))
        recall = true_pos/(true_pos+false_neg)
        precision = true_pos/(true_pos+false_pos)
        if(recall+precision==0):
            f1_scores.append(0)
            continue
        f1 = 2*(recall*precision)/(recall+precision)
        f1_scores.append(f1)
    return np.mean(f1_scores)
        
  

In [3]:
class KNN:
    def __init__(self,neighbour=2):
        self.neighbour = neighbour

    def cell_dector(self,row):
        count = 0
        for cell in self.missing_index:
            if(cell[0]==row):
                count = count+1
        return count
        
    def fill(self,dataset):
        self.dataset = dataset.copy()
        self.missing_index = np.argwhere(pd.isna(self.dataset))
        #print(self.missing_index)
        self.col = dataset.shape[1]
        distances = []
        for missing_cell in self.missing_index:
            unfilled_cell = self.cell_dector(missing_cell[0])
            filled_cell = self.col-unfilled_cell
            distance = []
            for i in range(self.dataset.shape[0]):
                if(i==missing_cell[0]):
                    continue
                row_a = self.dataset.iloc[missing_cell[0]]
                row_b = self.dataset.iloc[i]
                
                filled_cell_check = ~(row_a.isna() | row_b.isna())
                filled_cell_check.iloc[missing_cell[1]]=False
                perfect_a_row = row_a[filled_cell_check]
                perfect_b_row = row_b[filled_cell_check]

                if len(perfect_a_row) == 0:
                    continue
                
                dis_cal = np.sqrt((filled_cell/self.col)*np.sum((perfect_a_row-perfect_b_row)**2))
                distance.append((i,dis_cal))
            minimum_index = self.index_finder(distance)
            
            sum = 0
            check = 0
            for alok in minimum_index:
                if(pd.isna(self.dataset.iloc[alok,missing_cell[1]])):
                    continue
                    
                sum = sum+self.dataset.iloc[alok,missing_cell[1]]
                check = check+1
            if (check>0):
                fill_values = sum/check
                self.dataset.iloc[missing_cell[0],missing_cell[1]]=fill_values
            
        return self.dataset 
    def index_finder(self, distance):
        index = []
        value = []
        for dist in distance:
            index.append(dist[0])
            value.append(dist[1])
        value = np.array(value)
        index = np.array(index)
        minimum_value = value.argsort()
        minimum_index = index[minimum_value][:self.neighbour]
        return minimum_index


In [4]:
class Scaler:
    def __init__(self):
        self.mean = None
        self.std = None
        
    def scale(self,dataset):
        
        self.mean = dataset.mean()
        self.std = dataset.std()
        
    
    def transform(self,dataset):
        new_dataset = (dataset-self.mean)/(self.std)
        return new_dataset

In [5]:
class Train_test_split:
    def __init__(self,dataset,test_size=0.2,random=1,target='target'):
        self.target = target
        self.random_state=random
        self.test_size=test_size
        self.dataset = dataset
        
        self.row = self.dataset.shape[0]


    def split(self):
        
        np.random.seed(self.random_state)
        row_count = int(self.row*self.test_size)
        value = np.arange(0,self.row)
        row_no = np.random.choice(value,replace=False,size=row_count)
        

        test_dataset = self.dataset.iloc[row_no]
        x_test = test_dataset.drop(columns=[self.target])
        y_test = test_dataset[self.target]

        remaining_row=[]
        for i in range(self.row):
            if i not in row_no:
                remaining_row.append(i)
        train_dataset = self.dataset.iloc[remaining_row]
        x_train = train_dataset.drop(columns=[self.target])
        y_train = train_dataset[self.target]
        
        return x_train,x_test,y_train,y_test
        
                


# **For Multiclass classification**

In [6]:
data = pd.read_csv('train_nn.csv')

data = data.iloc[:,1:]

target = data['target_cls']
data.drop(columns=['target_reg','target_cls'],inplace=True)
s=Scaler()
s.scale(data)
data = s.transform(data)

In [7]:
n = Nueral_Network(data,target,weight_init='He',nueron=[64,32,26],opti='Adam')
test_data = pd.read_csv('test_nn.csv')
test_data=test_data.iloc[:,1:]
test_data = s.transform(test_data)
predicted_class=n.predict(test_data)

Loss after 0 iteration is 4.746548088365848
Loss after 100 iteration is 0.24547402914398658
Loss after 200 iteration is 0.09978452561327279
Loss after 300 iteration is 0.043613263111143635
Loss after 400 iteration is 0.016821918126348513
Loss after 500 iteration is 0.006267465602925876
Loss after 600 iteration is 0.002515174642306827
Loss after 700 iteration is 0.0011499577682820097
Loss after 800 iteration is 0.0005951071862565018
Loss after 900 iteration is 0.000350424177948912
Final loss is 0.00023176795254145928

        (\___/)
        (^ _ ^)
        (>Training complete💕
        


In [42]:
data = pd.read_csv('train_nn.csv')

data = data.iloc[:,1:]

In [72]:
data['target_cls'].nunique()

26

In [9]:
id = np.arange(test_data.shape[0])
nn_submit=pd.DataFrame({
    'id':id,
    'Target_cls':predicted_class
})

In [10]:
nn_submit.to_csv('nn_multi_submit.csv',index=False)

# **Regression Dataset**

In [11]:
data = pd.read_csv('train_nn.csv')

data = data.iloc[:,1:]

target = data['target_reg']
data.drop(columns=['target_reg','target_cls'],inplace=True)
s=Scaler()
s.scale(data)
data = s.transform(data)



In [12]:
n = Nueral_Network(data,target,weight_init='He',type='regression',nueron=[512,256,128,64,1],opti='Adam',epoch=3400)
test_data = pd.read_csv('test_nn.csv')
test_data=test_data.iloc[:,1:]
test_data = s.transform(test_data)
n.predict(data.iloc[:4])

Loss after 0 iteration is 61.98043340825482
Loss after 100 iteration is 1.6377373314529364
Loss after 200 iteration is 0.15805772630925446
Loss after 300 iteration is 0.038372038461542406
Loss after 400 iteration is 0.09069461360403135
Loss after 500 iteration is 0.017023283055053195
Loss after 600 iteration is 0.00797879665820567
Loss after 700 iteration is 0.005273388898054108
Loss after 800 iteration is 0.006383458728148289
Loss after 900 iteration is 0.008168478217888273
Loss after 1000 iteration is 0.017829394941353313
Loss after 1100 iteration is 0.006577669801974116
Loss after 1200 iteration is 0.00467839731994173
Loss after 1300 iteration is 0.01381093148046481
Loss after 1400 iteration is 0.0053810488433984295
Loss after 1500 iteration is 0.0013608500912771651
Loss after 1600 iteration is 0.0010372377536383155
Loss after 1700 iteration is 0.0012727468341500253
Loss after 1800 iteration is 0.0077217256551946846
Loss after 1900 iteration is 0.016150508765656192
Loss after 2000 i

array([[7.3375999 ],
       [7.77183411],
       [6.2437877 ],
       [8.70726675]])

In [16]:
pre_y = n.predict(test_data)
pre_y = pre_y.ravel()
ids = np.arange(test_data.shape[0])

nn_submit = pd.DataFrame({
    'id': ids,
    'Target_reg': pre_y
})

nn_submit.to_csv('nn_regression_submit.csv', index=False)

## **For Multi-Class classification-2**

In [75]:
data3 = pd.read_csv('train_multi_class_new.csv')

In [76]:
target = data3.iloc[:,-1]

In [77]:
t = Train_test_split(data3)
x_train,x_test,y_train,y_test = t.split()

In [78]:
s = Scaler()
s.scale(x_train)
x_train=s.transform(x_train)
x_test=s.transform(x_test)

In [79]:
unique_class = np.unique(data3['target']).shape[0]
unique_class

6

In [80]:
n = Nueral_Network(x_train,y_train,nueron=[32,16,8,unique_class],learning_rate=0.1,epoch=3000)

Loss after 0 iteration is 2.3528074061507036
Loss after 100 iteration is 0.8747077614900789
Loss after 200 iteration is 0.6886818967241356
Loss after 300 iteration is 0.631408280472146
Loss after 400 iteration is 0.6008635704139007
Loss after 500 iteration is 0.5806935659340768
Loss after 600 iteration is 0.5680965332845108
Loss after 700 iteration is 0.5539760410303776
Loss after 800 iteration is 0.5511649244465784
Loss after 900 iteration is 0.5409608500093638
Loss after 1000 iteration is 0.5361869782097468
Loss after 1100 iteration is 0.5330363449844404
Loss after 1200 iteration is 0.5374068222349391
Loss after 1300 iteration is 0.5290647196735095
Loss after 1400 iteration is 0.5261312349644047
Loss after 1500 iteration is 0.5220712586280108
Loss after 1600 iteration is 0.5189644125236214
Loss after 1700 iteration is 0.5170582710965527
Loss after 1800 iteration is 0.5141840188425805
Loss after 1900 iteration is 0.5116177034915318
Loss after 2000 iteration is 0.5097962037777694
Loss 

In [63]:
pre_value = n.predict(x_test)

In [81]:
score = macro_f1(pre_value,y_test)

In [83]:
score

np.float64(0.8153533169505064)